In [1]:
import nltk
!pip3 install wikipedia-api
import wikipediaapi
from collections import defaultdict
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import numpy as np
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import gutenberg
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
corpus_data=list(nltk.corpus.gutenberg.words('austen-emma.txt'))
with open("/content/Analogy_dataset.txt", "r") as f:
    lines = f.read().splitlines()
analogies=[]
for i in range(len(lines)):
    analogies +=lines[i].split(" ")
analogies=analogies[:-1]

In [3]:
def get_k_sentences(word,k):
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)

    p_wiki = wiki_wiki.page(word)
    txt=p_wiki.text
    sentences = txt.split(".")
    k_sentences = []
    for sent in sentences:
        check_word = sent.split(" ")
        if word in check_word and len(check_word) < 50:
            k_sentences.append(sent)
        if len(k_sentences) > k:
            break
    return k_sentences

In [4]:
def tokenised_additional_corpus(k):
    analogy_raw_corpus = []
    for analogy in analogies:
      if analogy != " ":
          analogy_raw_corpus += get_k_sentences(analogy,k)
    full_analogy_corpus = ' '.join([elem for elem in analogy_raw_corpus])
    data = re.sub(r'[,!?;-]+\n:', '.', full_analogy_corpus)
    data = nltk.word_tokenize(data)
    return data

In [6]:
def full_corpus(data,stemmer):
    corpus=data
    stop_words = set(stopwords.words('english'))
    pattern = re.compile("^[a-zA-Z]+$")
    porter = PorterStemmer()
    data=[]
    for ch in corpus:
        ch = ch.lower()
        if pattern.match(ch) and ch not in stop_words and ch != '.':
                if stemmer==True:
                     data.append(porter.stem(ch))
                else:
                     data.append(ch)
    return data

get full corpus data (after performing pre processing) = gutenburg corpus + k tokenized sentences

In [7]:
processed_corpus = full_corpus(tokenised_additional_corpus(5),False)+full_corpus(corpus_data,True) #initailly assuming k=6 to check

In [16]:
covab_corpus=list(sorted(set(processed_corpus)))

In [18]:
len(covab_corpus)

6698

In [19]:
freq_dist = nltk.FreqDist(word for word in processed_corpus)
print("Total words in corpus: ",len(processed_corpus) )
print("Size of vocabulary: ",len(freq_dist) )
print("Most frequent tokens: ",freq_dist.most_common(20) )
V=len(freq_dist)

Total words in corpus:  85757
Size of vocabulary:  6698
Most frequent tokens:  [('mr', 1852), ('emma', 865), ('could', 838), ('would', 826), ('miss', 611), ('must', 568), ('one', 516), ('harriet', 506), ('much', 494), ('said', 487), ('think', 466), ('thing', 460), ('weston', 448), ('everi', 435), ('well', 415), ('elton', 407), ('knightley', 396), ('know', 393), ('say', 392), ('good', 365)]


In [20]:
def indices_info(processed_corpus):
    processed = sorted(list(set(processed_corpus)))
    index = 0
    wordxInd = {}
    Indxword = {}
    for word in processed:
        if word not in wordxInd: 
             wordxInd[word] = index
             Indxword[index] = word
             index += 1
    return wordxInd, Indxword

In [22]:
covab_corpus[0:5]

['abbey', 'abbot', 'abdi', 'abdul', 'abhor']

In [23]:
import itertools
wordxInd, Indxword = indices_info(processed_corpus)
print(dict(itertools.islice(wordxInd.items(), 5)))
print(dict(itertools.islice(Indxword.items(), 5)))

{'abbey': 0, 'abbot': 1, 'abdi': 2, 'abdul': 3, 'abhor': 4}
{0: 'abbey', 1: 'abbot', 2: 'abdi', 3: 'abdul', 4: 'abhor'}


In [28]:
vocabulary =covab_corpus
import pandas as pd
df = pd.DataFrame({'vocab': vocabulary})
one_hot_encoded = pd.get_dummies(df['vocab'])

In [37]:
one_hot_encoded

,abbey,abbot,abdi,abdul,abhor,abid,abil,abjad,abl,abod,...,youth,yugoslavia,zamora,zeal,zealand,zigzag,zimbabwe,zone,zones,zoroastrianism
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [39]:
for x, y in get_windows(processed_corpus[2000:2010],3):
    print(f'{x}\t{y}')

['people', 'also', 'part', 'urban', 'agglomeration', 'africa']	largest
['also', 'part', 'largest', 'agglomeration', 'africa', 'arab']	urban
['part', 'largest', 'urban', 'africa', 'arab', 'world']	agglomeration
['largest', 'urban', 'agglomeration', 'arab', 'world', 'middle']	africa


In [40]:
one_hot_encoded['though'].to_numpy().reshape

<function ndarray.reshape>

In [42]:
def get_train_set():
   train_x =None
   train_y = None
   
   for context_words, center_word in get_windows(processed_corpus[0:5000], 3):
      center_word_vector=one_hot_encoded[center_word].to_numpy()
      center_word_vector=np.resize(center_word_vector,(center_word_vector.shape[0],1))
      context_words_vector=0  
      for word in context_words:      
           context_words_vector += one_hot_encoded[word].to_numpy()
      context_words_vector=np.resize(context_words_vector,(context_words_vector.shape[0],1))
      if train_x is not None:
           train_x=np.append(train_x, context_words_vector/len(context_words), axis=1)
           train_y=np.append(train_y, center_word_vector, axis=1)
      else:
           train_x=context_words_vector/len(context_words)
           train_y=center_word_vector

   return train_x,train_y

In [43]:
x,y= get_train_set()


In [44]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
def initialize_model(N,V, random_seed=1):
    np.random.seed(random_seed)
    W1 = np.random.rand(N,V)
    W2 = np.random.rand(V,N)
    b1 = np.random.rand(N,1)
    b2 = np.random.rand(V,1)
    return W1, W2, b1, b2

In [46]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z,axis=0,keepdims=True)
    return e_z/sum_e_z

In [47]:
def forward_prop(x, W1, W2, b1, b2):
    h = np.dot(W1,x) + b1
    h = np.maximum(0,h)
    z = np.dot(W2,h) + b2
    return z, h

In [48]:
def compute_cost(y, yhat):
    m=y.shape[1]
    logprobs = np.multiply(np.log(yhat),y)
    cost = - 1/m * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [49]:
def back_prop(x, yhat, y, h, W1, W2, b1, b2):
    m=x.shape[1]
    l1 = np.dot(W2.T ,(yhat - y))
    l1 = np.maximum(0,l1)
    grad_W1 = np.dot(l1,x.T)/m
    grad_W2 = np.dot((yhat - y),h.T)/m
    grad_b1 = np.sum(l1,axis=1,keepdims=True)/m
    grad_b2 = np.sum((yhat - y),axis=1,keepdims=True)/m
    return grad_W1, grad_W2, grad_b1, grad_b2

In [ ]:
#x, y = get_train_set()

In [50]:
def gradient_descent(N, num_iters,alpha, 
                     random_seed, initialize_model=initialize_model, 
                     forward_prop=forward_prop, 
                     softmax=softmax,
                     compute_cost=compute_cost, 
                     back_prop=back_prop):
  
    W1, W2, b1, b2 = initialize_model(N,V, random_seed=random_seed)
    iters = 0
    for i in range(num_iters+1):
        z, h = forward_prop(x, W1, W2, b1, b2)
        yhat = softmax(z)
        cost = compute_cost(y, yhat)
        if ( (iters+1) % 10 == 0):
            print(f"iters: {iters + 1} cost: {cost:.6f}")
        grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(x, yhat, y, h, W1, W2, b1, b2)

        W1 = W1 - alpha * grad_W1
        W2 = W2 - alpha * grad_W2
        b1 = b1 - alpha * grad_b1
        b2 = b2 - alpha * grad_b2
        iters +=1 
        if iters == num_iters: 
            break
            
    return W1, W2, b1, b2

In [52]:
W1, W2, b1, b2=gradient_descent(500, 210,0.06,1)

iters: 10 cost: 17.749115
iters: 20 cost: 15.501379
iters: 30 cost: 13.941503
iters: 40 cost: 12.746420
iters: 50 cost: 11.804557
iters: 60 cost: 11.058228
iters: 70 cost: 10.458509
iters: 80 cost: 9.969619
iters: 90 cost: 9.572253
iters: 100 cost: 9.253129
iters: 110 cost: 9.007025
iters: 120 cost: 8.827385
iters: 130 cost: 8.710967
iters: 140 cost: 8.654843
iters: 150 cost: 8.646518
iters: 160 cost: 8.671101
iters: 170 cost: 8.713344
iters: 180 cost: 8.758393
iters: 190 cost: 8.794323
iters: 200 cost: 8.817982
iters: 210 cost: 8.830083


In [53]:
W1

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, ...,
        2.85312388e-01, 8.71852190e-01, 4.71602282e-02],
       [3.64310657e-01, 7.74035414e-01, 4.85513882e-01, ...,
        1.79367285e-01, 8.80412834e-01, 2.19778595e-01],
       [5.17081475e-01, 6.80846766e-01, 3.29858347e-01, ...,
        6.04752846e-01, 5.46033231e-01, 9.42977127e-01],
       ...,
       [4.81957834e-01, 4.71676473e-01, 8.80020169e-01, ...,
        5.14653681e-01, 7.18061055e-01, 6.73145456e-01],
       [2.01923343e-01, 6.53384118e-01, 4.41259750e-01, ...,
        2.86217691e-01, 8.05183709e-01, 5.93183564e-01],
       [1.75523836e-01, 8.13210086e-01, 6.87718710e-01, ...,
        9.83985406e-01, 7.65344714e-01, 9.70616300e-01]])

In [152]:
def euclidean(A, B):
   
    d = np.linalg.norm(A-B)


    return d

In [153]:
def cosine_similarity(A, B):
 

    dot = np.dot(A,B)  
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot/(norma*normb)

    return cos

In [155]:
def get_analogy(word1, word2, word3, embeddings, cosine_similarity=cosine_similarity):

    group=[word1.lower(), word2.lower(), word3.lower()]
    word1_emb = embeddings[wordxInd[word1.lower()]]
    word2_emb = embeddings[wordxInd[word2.lower()]]
    word3_emb = embeddings[wordxInd[word3.lower()]]

    # Remember: King - Man + Woman = Queen
    vec =  word2_emb - word1_emb + word3_emb

    similarity = -1

    country = ''

    for word in covab_corpus:

        if word not in group :

            # get the word embedding
            word_emb = embeddings[wordxInd[word.lower()]]

            cur_similarity = cosine_similarity(word_emb,vec)

            if cur_similarity > similarity:

                similarity = cur_similarity

                country = (word,similarity)

    return country

In [156]:
embs = (W1.T+W2)/2

In [158]:
get_analogy("Maharastra","Mumbai", "Karnataka", embs, cosine_similarity=cosine_similarity)

('cosmopolitan', 0.7657802968477159)

In [159]:
x.shape

(6698, 4994)